# Temporal Fusion Transformer on Boeing Stock Prices
This notebook demonstrates how to load, preprocess, and train a TFT model using PyTorch Forecasting.

In [ ]:
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_forecasting.metrics import QuantileLoss
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer

## 1. Load and Preprocess Data

In [ ]:
data = pd.read_csv("Data collected - Boeing stock price.csv", usecols=[0, 1])
data.columns = ["Date", "Close"]
data["Date"] = pd.to_datetime(data["Date"], errors="coerce")
data = data.dropna(subset=["Date"])
data = data.sort_values("Date")
data["time_idx"] = range(len(data))
data["group_id"] = "BOEING"
data["target"] = data["Close"]
data.head()

## 2. Create TimeSeriesDataSet

In [ ]:
training = TimeSeriesDataSet(
    data,
    time_idx="time_idx",
    target="target",
    group_ids=["group_id"],
    max_encoder_length=30,
    max_prediction_length=7,
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=[],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["target"],
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

In [ ]:
train_dataloader = training.to_dataloader(train=True, batch_size=64)

## 3. Build TFT Model

In [ ]:
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
    log_val_interval=1,
)

## 4. Train Model

In [ ]:
trainer = Trainer(
    max_epochs=30,
    accelerator="cpu"
)
trainer.fit(tft, train_dataloader)